In [ ]:
# This notebook was originally written by Søren Fomsgaard, july 2020.

Here is my extended search function fromte text processing assignment

In [2]:
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

%load_ext jupyternotify

# test
#%notify
#import time
#time.sleep(5)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
from notify_run import Notify

In [3]:
notify = Notify()

In [6]:
notify.register()

Endpoint: https://notify.run/Nlv7lmoz5ACPDRHz
To subscribe, open: https://notify.run/c/Nlv7lmoz5ACPDRHz
Or scan this QR code:

                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              

In [4]:
import re, glob, os
from collections import defaultdict
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
import math

In [25]:
# ver. 3 - debugging the count of regexes.

### NOTE ###
# - The code counts terms twice, if they are similar within the same pass. This means that drunk* and drun* count 1 and 2, because the loop counts drunk* again on the second iteration. THIS IS A PROBLEM WITH DRUNK* AND DRUNKARD!!!
# -  

def regex_search(folder_path , regList):
    """
    Search a folder path for a given regex.

    :folder_path: a string with the path to a dir with processed .txt files 
    :regList: a list of regular expressions as strings.
    """
    resultsDict = defaultdict(lambda: defaultdict(dict)) #consider using regular dict for the inner layer, since you know which terms you are searching.


    # file_names = [] 

    # compile any search term(s) given for searching.
    #search_terms = re.compile('|'.join(regList)) # with word boundaries: re.compile(r'\b(?:%s)\b' % '|'.join(regList))
    search_terms_re = [re.compile(reg) for reg in regList]

        
    #loop through the folder with txt files.
    
    for filepath in tqdm(list(glob.iglob(folder_path + "*\\*", recursive=True))):  #consider using os.listdir() for a subset of files
        filename = os.path.basename(filepath)
        
        
        #open the file
        with open(filepath, "r") as infile:
            content = infile.read()

        #find the document length (total tokens)
        resultsDict[filename]['text_length'] = len(re.findall(r'\w+', content)) #consider using the token pattern from https://github.com/gearmonkey/tfidf-python/blob/master/tfidf.py : re.findall(r"<a.*?/a>|<[^\>]*>|[\w'@#]+")

        #search for the terms in the file, given the list of regexes.
        for term in search_terms_re:
            n_hits = 0
            for match in re.finditer(term, content):
                #print(term, match)
                n_hits += 1
                #print(n_hits)

        # add the count of hits to the nested results dictionary with readable versions of the terms. 
                resultsDict[filename][re.sub(r'\\.*$', '', str(term.pattern))] = n_hits 



    #export to df
    resultsDF = pd.DataFrame.from_dict(resultsDict, orient='index').fillna(0)
    # (re)name the index column
    resultsDF = resultsDF.rename_axis('file_id')
    # sum counts for the term columns
    resultsDF['terms_sum'] = resultsDF.iloc[:, 1:].sum(axis=1)  # https://stackoverflow.com/questions/48923460/how-do-i-sum-a-column-range-in-python
    # trim output to only include docs with hits from the search
    trimmedOutput = resultsDF[(resultsDF['terms_sum'] > 0)]

    # compute number of hits (in terms of docs) / total corpus size
    percentageHits = int((len(trimmedOutput) / len(resultsDF)) * 100)
    #print(f'percentage of files found in the total corpus on this query:  {percentageHits}%')
    
    # Add a bottom row with totals for all columns
    trimmedOutput.loc[f'Total ({len(trimmedOutput)} doc(s) = {percentageHits}% of corpus)'] = trimmedOutput.sum(numeric_only=True, axis=0)
    
    # return both the raw output and the trimemd output containing only the files with hits.
    return resultsDF, trimmedOutput

#print(output)
#print(list(results.items())[:10])


#### for this code I used solutions from:
# https://stackoverflow.com/questions/6750240/how-to-do-re-compile-with-a-list-in-python
# https://howchoo.com/g/yjjknjdinmq/nested-defaultdict-python
# https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index 
# https://www.geeksforgeeks.org/python-program-to-count-words-in-a-sentence/


In [26]:
searchQuery = [
                "drunk\w*",
                "intoxicat\-?\w*",
                "temperance",
                "beer\-?\w*",
                "liquor",
                "alcohol\-?\w*",
                "brandy",
                "booz\-?\w*",
                "drunkard",
                "by\-drink\w*"]

In [27]:

#function call

rawOutput, trimmedOutput = regex_search("C://TEMPORARY/fullOB/" , searchQuery)
%notify

100%|█████████████████████████████████████████████████████████████████████████| 202931/202931 [16:53<00:00, 200.20it/s]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<IPython.core.display.Javascript object>

In [28]:
# inspect the trimmed output
trimmedOutput

,text_length,beer,drunk,brandy,liquor,intoxicat,booz,drunkard,temperance,alcohol,terms_sum
file_id,,,,,,,,,,,
1676-01-17_o16760117-1-defend16.txt,1106.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1676-04-05_t16760405-10-victim20.txt,147.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1676-08-23_t16760823-4-punish14.txt,275.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1676-12-13_t16761213-5-verdict15.txt,88.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1677-06-01_t16770601-1-verdict3.txt,135.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1913-04-01_t19130401-38-punishment-46.txt,690.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1913-04-01_t19130401-45-verdict-2.txt,1169.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1913-04-01_t19130401-46-punishment-51.txt,705.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [39]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('multiple_results.xlsx', engine='xlsxwriter')

In [40]:
# write the TRIMMED  output (only containing search hits) to a human friendly files

trimmedOutput.to_csv('trimmed_results.tsv', sep="\t") # index_label='file_id' 
trimmedOutput.to_excel(writer, sheet_name='trimmed_results')
rawOutput.to_excel(writer, sheet_name='raw_results')
writer.save()

## Calculating TF-IDF for the search results
Next, we load the data, and calculate the tf-df for each term per doc, and find their averages per doc.

In [131]:
# read the output of the last run, drop the index column and the footer (containing sum totals).
# consider reading multiple excel sheets.
xls_in = pd.ExcelFile('multiple_results.xlsx')
df_in = pd.read_excel(xls_in, 'trimmed_results', index_col=False, skipfooter=1) #read the trimmed results without the footer
df_in.set_index('file_id' , inplace=True) # drop the added index.
del df_in['terms_sum'] # remove the totals for all terms taken together.

KeyboardInterrupt: 

In [70]:
# inspect the loaded data

df_in

,text_length,beer,drunk,brandy,liquor,intoxicat,booz,drunkard,temperance,alcohol
file_id,,,,,,,,,,
1676-01-17_o16760117-1-defend16.txt,1106,1,0,0,0,0,0,0,0,0
1676-04-05_t16760405-10-victim20.txt,147,1,0,0,0,0,0,0,0,0
1676-08-23_t16760823-4-punish14.txt,275,0,1,0,0,0,0,0,0,0
1676-12-13_t16761213-5-verdict15.txt,88,0,1,0,0,0,0,0,0,0
1677-06-01_t16770601-1-verdict3.txt,135,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1913-04-01_t19130401-34-punishment-44.txt,933,1,0,0,0,0,0,0,0,0
1913-04-01_t19130401-38-punishment-46.txt,690,2,2,0,0,0,0,0,0,0
1913-04-01_t19130401-45-verdict-2.txt,1169,5,0,0,0,0,0,0,0,0


In [132]:
df_tfidf = pd.DataFrame # index= the header in df_in , colum should be tf , idf , tf-idf, average (tfidf) <- this is basically just flipping part of the other dataframe around on all rows, colum 1:-1. transpose? 

# tf = df_in[['text_length' , 'drunk*']].sum(axis=1)
normalized_tf = df_in.iloc[:, 1:].divide(df_in['text_length'] , axis=0)


#idf = len(df_in).divide(df_in.iloc[:, 1:] , axis=0) ## put this in the new df for each row

#idf

normalized_tf = normalized_tf.transpose()
normalized_tf
# consider writing a function for each variable here and using .apply to insert the resulting computations in new columns.
# otherwise, load it back into another container (dict of dicts and compute things from there before moving back to dataframe)

#idf = len(df_in['file_id'] / column_values >=1) ### total documents / total document with term t in them.
#df_in['tf_idf'] = tf*idf

file_id,1676-01-17_o16760117-1-defend16.txt,1676-04-05_t16760405-10-victim20.txt,1676-08-23_t16760823-4-punish14.txt,1676-12-13_t16761213-5-verdict15.txt,1677-06-01_t16770601-1-verdict3.txt,1677-06-01_t16770601-4-verdict15.txt,1678-12-11_16781211.txt,1680-01-17_t16800117-1-verdict4.txt,1680-09-10_s16800910a-1.txt,1680-09-10_t16800910-6-punish33.txt,...,1913-02-04_t19130204-74-verdict-1.txt,1913-03-04_t19130304-14-verdict-1.txt,1913-03-04_t19130304-39-punishment-39.txt,1913-03-04_t19130304-48-punishment-52.txt,1913-03-04_t19130304-54-punishment-57.txt,1913-04-01_t19130401-34-punishment-44.txt,1913-04-01_t19130401-38-punishment-46.txt,1913-04-01_t19130401-45-verdict-2.txt,1913-04-01_t19130401-46-punishment-51.txt,1913-04-01_t19130401-47-punishment-52.txt
beer,0.000904,0.006803,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000692,0.005,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.001072,0.002899,0.004277,0.000000,0.001767
drunk,0.000000,0.000000,0.003636,0.011364,0.007407,0.002092,0.000288,0.001052,0.000000,0.000,...,0.000000,0.003236,0.003781,0.0000,0.000328,0.000000,0.002899,0.000000,0.002837,0.001767
brandy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
liquor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.0002,0.000082,0.000000,0.000000,0.000000,0.000000,0.000000
intoxicat,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000767,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
booz,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
drunkard,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.0000,0.000164,0.000000,0.000000,0.000000,0.000000,0.000000
temperance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
alcohol,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Stella's function

In [33]:
frequency_results = pd.read_excel('multiple_results.xlsx', sheet_name='trimmed_results', index_col=False).set_index('file_id' , inplace=True)
df_results = pd.read_excel('multiple_results.xlsx', sheet_name='trimmed_results', index_col=False, skipfooter=1).set_index('file_id' , inplace=True) #for using the function below
raw_search = pd.read_excel('multiple_results.xlsx', sheet_name='raw_results', index_col=False).set_index('file_id' , inplace=True)

In [36]:
df_results

In [29]:

#df_results = pd.read_excel("freq_dict.xlsx")

def get_tf_idf(final_df, freq_df, term):
    """
    Blablabla
    """
    
    final_df[term+'_freq'] = freq_df[term]
    final_df['tf_'+term] = (final_df[str(term+'_freq')] / final_df['text_length'])
    
    counter = 0
    for index, freq in enumerate(final_df[term+'_freq']):
        if final_df.iloc[index][term+'_freq'] != 0:
            counter+= 1
        
    final_df['idf_'+term] = (math.log((len(final_df['file_id']))/ counter))
    
    final_df['tf.idf_'+term] = final_df['tf_'+term] * final_df['idf_'+term]
    
    columns = ['tf_'+term, 'idf_'+term, term+'_freq']
    final_df = final_df.drop(columns, axis=1)
    return(final_df)

final_df = pd.DataFrame()
final_df['file_id'] = df_results['file_id']
final_df['text_length'] = df_results['text_length']

freq_df = df_results

final_df = get_tf_idf(final_df, freq_df, 'drunk')
final_df = get_tf_idf(final_df, freq_df, 'beer')
final_df = get_tf_idf(final_df, freq_df, 'intoxicat')
final_df = get_tf_idf(final_df, freq_df, 'temperance')
final_df = get_tf_idf(final_df, freq_df, 'liquor')
final_df = get_tf_idf(final_df, freq_df, 'alcohol')
final_df = get_tf_idf(final_df, freq_df, 'brandy')
final_df = get_tf_idf(final_df, freq_df, 'booz')
final_df = get_tf_idf(final_df, freq_df, 'drunkard')
#final_df = get_tf_idf(final_df, freq_df, 'by-drink') # the search never finds this!

# find the averages
final_df['average_tf.idf'] = (final_df.iloc[: ,3 :].sum(axis=1) / len((final_df.iloc[: ,3 :].columns)))

#final_df.to_excel('tf_idf_Soren2.xlsx', index_label='file_id')

In [30]:
final_df

,file_id,text_length,tf.idf_drunk,tf.idf_beer,tf.idf_intoxicat,tf.idf_temperance,tf.idf_liquor,tf.idf_alcohol,tf.idf_brandy,tf.idf_booz,tf.idf_drunkard,average_tf.idf
0,1676-01-17_o16760117-1-defend16.txt,1106,0.000000,0.000650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000081
1,1676-04-05_t16760405-10-victim20.txt,147,0.000000,0.004890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000611
2,1676-08-23_t16760823-4-punish14.txt,275,0.003577,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,1676-12-13_t16761213-5-verdict15.txt,88,0.011179,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,1677-06-01_t16770601-1-verdict3.txt,135,0.007287,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
28151,1913-04-01_t19130401-34-punishment-44.txt,933,0.000000,0.000770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000096
28152,1913-04-01_t19130401-38-punishment-46.txt,690,0.002851,0.002083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000260
28153,1913-04-01_t19130401-45-verdict-2.txt,1169,0.000000,0.003074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000384
28154,1913-04-01_t19130401-46-punishment-51.txt,705,0.002791,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [20]:
final_df.iloc[: ,3 :].mean(axis=1) # the statistical mean

0        0.000081
1        0.000611
2        0.000000
3        0.000000
4        0.000000
           ...   
28151    0.000096
28152    0.000260
28153    0.000384
28154    0.000000
28155    0.000159
Length: 28156, dtype: float64

In [28]:
average = (final_df.iloc[: ,3 :].sum(axis=1) / len((final_df.iloc[: ,3 :].columns)))
print(average)

0        0.000081
1        0.000611
2        0.000000
3        0.000000
4        0.000000
           ...   
28151    0.000096
28152    0.000260
28153    0.000384
28154    0.000000
28155    0.000159
Length: 28156, dtype: float64


In [31]:
writer = pd.ExcelWriter('final_results.xlsx', engine='xlsxwriter')

In [32]:
final_df.to_excel(writer, sheet_name='tf-idf_results', index_label='file_id' )
frequency_results.to_excel(writer, sheet_name='frequency_results')
raw_search.to_excel(writer, sheet_name='raw_results')
writer.save()